In [1]:

import torch
import pandas as pd
import shelve
import features
import nnue_dataset
from tqdm import tqdm
import logging 
import numpy as np
from scipy.sparse import lil_matrix, vstack
import click
import os
import pickle as pkl
import model_single_bucket as M


def get_data(feature_set, fens):
    b = nnue_dataset.make_sparse_batch_from_fens(
        feature_set, fens, [0] * len(fens), [1] * len(fens), [0] * len(fens)
    )
    return b

def get_single_activation(model, forward_mode, feature_set, fens):

    model.forward_mode = forward_mode

    b = get_data(feature_set, fens)
    
    (
        us,
        them,
        white_indices,
        white_values,
        black_indices,
        black_values,
        outcome,
        score,
        psqt_indices,
        layer_stack_indices,
    ) = b.contents.get_tensors("cuda")
    out = model.forward(
        us,
        them,
        white_indices,
        white_values,
        black_indices,
        black_values,
        psqt_indices,
        layer_stack_indices,
    )
    out_numpy = out.cpu().detach().numpy()

    del out    


    return out_numpy

In [2]:

features_name = "HalfKAv2_hm^"
feature_set = features.get_feature_set_from_name(features_name)

nnue = M.NNUE(feature_set)
nnue.load_from_checkpoint('/home/ap/nnue-pytorch/logs/lightning_logs/version_19_setting_0/checkpoints/epoch=499-step=3052000.ckpt', feature_set=feature_set)
nnue.cuda()

NNUE(
  (input): DoubleFeatureTransformerSlice()
  (layer_stacks): LayerStacks(
    (l1): Linear(in_features=2048, out_features=128, bias=True)
    (l2): Linear(in_features=128, out_features=128, bias=True)
    (output): Linear(in_features=128, out_features=1, bias=True)
    (relu): ReLU()
  )
)

In [1]:
import pandas as pd
db_name = "stockfish_data_fens_01"
dir_name = f"/media/ap/storage/stockfish_data/{db_name}"

# load df
df = pd.read_pickle(f"{dir_name}/concept_df.pkl")

In [4]:
df.to_pickle(f"production_models/fen_df.pkl")

In [2]:
df.head()

,fen,r,k,K,q,P,R,p,b,B,n,N,Q
0,8/8/8/2r5/1k6/5K2/8/6q1 b - - 7 78,1.0,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6k1/4RrP1/8/6K1/3Pp3/3b4/8/8 w - - 5 87,1.0,1,1,0.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
2,r4k2/2R1p2p/4b1p1/8/2P3P1/2b1B2P/4BPK1/8 b - -...,1.0,1,1,0.0,4.0,1.0,3.0,2.0,2.0,0.0,0.0,0.0
3,1r3k2/5ppp/3P4/6P1/3pP3/7B/2r2P2/3R1K2 w - - 0 36,2.0,1,1,0.0,4.0,1.0,4.0,0.0,1.0,0.0,0.0,0.0
4,8/1r4p1/k4p2/8/p6P/3K4/5P2/6R1 w - - 0 52,1.0,1,1,0.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0


In [14]:
i = 2
forward_mode = 3
fens = [df.iloc[i].fen]

In [15]:
out = get_single_activation(nnue, forward_mode, feature_set, fens)

In [16]:
out.shape

(1, 128)